Importing Libraries

In [7]:
# Imports and basic configuration

# Core utilities
import os
import random
import warnings

# Numerics and dataframes
import numpy as np
import pandas as pd

# Modeling utilities
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Set global random seed for reproducibility
RANDOM_STATE = 42
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# Configure pandas display for better readability in notebooks
pd.set_option('display.width', 140)
pd.set_option('display.max_columns', 120)

# Suppress unnecessary warnings during execution
warnings.filterwarnings('ignore')

# Define base directory for data files
DATA_DIR = "."

# Define full paths for key data files
PBP_FILE = os.path.join(DATA_DIR, "nfl_2024_new.csv")
SCORES_FILE = os.path.join(DATA_DIR, "nfl_2024_scores.csv")
UPCOMING_FILE = os.path.join(DATA_DIR, "upcoming_games_2025_week1.csv")

Scraping/Merging Data

In [8]:
# Data ingestion and merging

# Load 2024 play-by-play data (Weeks 1–17 only)
pbp_df = pd.read_csv(PBP_FILE)

# Load 2024 final scores for all games
scores_df = pd.read_csv(SCORES_FILE)

# Merge 1: Attempt to join by matching Offense vs. Defense to Visitor vs. Home
merged = pbp_df.merge(
    scores_df,
    left_on=['Date', 'OffenseTeam', 'DefenseTeam'],
    right_on=['Date', 'Visitor', 'Home'],
    how='left'
)

# Merge 2: Reverse the roles to catch games where Home/Visitor are flipped
merged = merged.merge(
    scores_df,
    left_on=['Date', 'OffenseTeam', 'DefenseTeam'],
    right_on=['Date', 'Home', 'Visitor'],
    how='left',
    suffixes=('', '_reverse')
)

# Fill missing columns using reverse match when needed
for col in ['Week', 'Visitor', 'VisitorScore', 'Home', 'HomeScore', 'OT']:
    merged[col] = merged[col].combine_first(merged[f"{col}_reverse"])

# Drop the redundant reverse columns
merged.drop(columns=[f"{col}_reverse" for col in ['Week', 'Visitor', 'VisitorScore', 'Home', 'HomeScore', 'OT']], inplace=True)

# Add binary target column: 1 if Home team won, 0 otherwise
merged['HomeWon'] = merged['HomeScore'] > merged['VisitorScore']

# Preview merged dataset
merged[['Date', 'Home', 'Visitor', 'HomeScore', 'VisitorScore', 'HomeWon']].head(50)

,Date,Home,Visitor,HomeScore,VisitorScore,HomeWon
0,12/29/2024,MIN,GB,27.0,25.0,True
1,12/29/2024,MIN,GB,27.0,25.0,True
2,12/29/2024,MIN,GB,27.0,25.0,True
3,12/29/2024,MIN,GB,27.0,25.0,True
4,12/29/2024,MIN,GB,27.0,25.0,True
5,12/29/2024,MIN,GB,27.0,25.0,True
6,12/29/2024,TB,CAR,48.0,14.0,True
7,12/29/2024,TB,CAR,48.0,14.0,True
8,12/29/2024,TB,CAR,48.0,14.0,True
9,12/29/2024,TB,CAR,48.0,14.0,True


Team Feature Extraction

In [9]:
# Feature engineering: basic scoring and win rate metrics

# Load schedule for upcoming games (Week 1 of 2025 season)
upcoming_games = pd.read_csv(UPCOMING_FILE)

# Preview the upcoming schedule to verify the structure
print(upcoming_games.head(16))

# --- Average Points Scored ---

# Calculate average points scored at home and away
avg_scored_home = merged['HomeScore'].groupby(merged['Home']).mean()
avg_scored_away = merged['VisitorScore'].groupby(merged['Visitor']).mean()

# Combine to get total average points scored per team
avg_points_scored = (avg_scored_home + avg_scored_away) / 2

# --- Average Points Allowed ---

# Calculate average points allowed at home and away
avg_allowed_home = merged['VisitorScore'].groupby(merged['Home']).mean()
avg_allowed_away = merged['HomeScore'].groupby(merged['Visitor']).mean()

# Combine to get total average points allowed per team
avg_points_allowed = (avg_allowed_home + avg_allowed_away) / 2

# --- Win Rate ---

# Count wins as home and away
home_wins = merged.groupby('Home')['HomeWon'].sum()
away_wins = merged.groupby('Visitor')['HomeWon'].apply(lambda x: len(x) - x.sum())

# Count total games played as home and away
home_games = merged['Home'].value_counts()
away_games = merged['Visitor'].value_counts()

# Total wins and games played
total_wins = home_wins + away_wins
total_games = home_games + away_games

# Calculate overall win rate
win_rate = total_wins / total_games

# --- Assemble team-level features ---

# Combine all features into one DataFrame
team_features = pd.DataFrame({
    'AvgPointsScored': avg_points_scored,
    'AvgPointsAllowed': avg_points_allowed,
    'WinRate': win_rate
})

# Reset index and name the index column explicitly
team_features.reset_index(names='Team', inplace=True)

# Preview the engineered features
team_features.head(32)


   Home Visitor
0   PHI     DAL
1   LAC      KC
2    NO     ARI
3   NYJ     PIT
4   IND     MIA
5   ATL      TB
6   WAS     NYG
7   JAX     CAR
8   CLE     CIN
9    NE      LV
10  SEA      SF
11  DEN     TEN
12   LA     HOU
13   GB     DET
14  BUF     BAL
15  CHI     MIN


,Team,AvgPointsScored,AvgPointsAllowed,WinRate
0,ARI,21.939050,22.417655,0.419609
1,ATL,22.178923,23.802673,0.504973
2,BAL,30.215613,22.331169,0.679041
3,BUF,32.132682,21.771034,0.810774
4,CAR,18.581203,31.259658,0.260912
5,CHI,17.595422,22.197156,0.244108
6,CIN,28.780905,26.494829,0.501461
7,CLE,15.724711,25.285987,0.185644
8,DAL,20.873660,27.726174,0.443215
9,DEN,24.825513,19.431284,0.565964


In [10]:
# Feature engineering: defensive metrics (conceded plays and turnovers)

# --- Conceded Plays ---

# Define a successful offensive play as either a touchdown or a non-turnover play
merged['SuccessfulPlay'] = merged['IsTouchdown'] | (~merged['IsInterception'] & ~merged['IsFumble'])

# Calculate average rate of successful plays conceded at home and as a visitor
avg_conceded_home = merged.groupby('Home')['SuccessfulPlay'].mean()
avg_conceded_away = merged.groupby('Visitor')['SuccessfulPlay'].mean()

# Combine to get overall conceded play rate per team
avg_conceded_plays = (avg_conceded_home + avg_conceded_away) / 2

# --- Forced Turnovers ---

# Define a turnover as either an interception or a fumble
merged['Turnover'] = merged['IsInterception'] | merged['IsFumble']

# Calculate average forced turnovers at home and as a visitor
avg_turnovers_home = merged.groupby('Home')['Turnover'].mean()
avg_turnovers_away = merged.groupby('Visitor')['Turnover'].mean()

# Combine to get overall forced turnover rate per team
avg_forced_turnovers = (avg_turnovers_home + avg_turnovers_away) / 2

# --- Merge Defensive Features ---

# Create defensive metrics dataframe
team_features_def = pd.DataFrame({
    'Team': team_features['Team'],
    'AvgPointsDefended': team_features['AvgPointsAllowed'],  # Already calculated earlier
    'AvgConcededPlays': avg_conceded_plays.values,
    'AvgForcedTurnovers': avg_forced_turnovers.values
})

# Join defensive features into existing feature set
team_features_combined = team_features.merge(team_features_def, on='Team', how='left')

# Preview combined features
team_features_combined.head(32)

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers
0,ARI,21.939050,22.417655,0.419609,22.417655,-1.019879,0.021251
1,ATL,22.178923,23.802673,0.504973,23.802673,-1.022210,0.023877
2,BAL,30.215613,22.331169,0.679041,22.331169,-1.015552,0.017282
3,BUF,32.132682,21.771034,0.810774,21.771034,-1.018904,0.020579
4,CAR,18.581203,31.259658,0.260912,31.259658,-1.017999,0.018779
5,CHI,17.595422,22.197156,0.244108,22.197156,-1.017277,0.017890
6,CIN,28.780905,26.494829,0.501461,26.494829,-1.019098,0.022027
7,CLE,15.724711,25.285987,0.185644,25.285987,-1.019512,0.021505
8,DAL,20.873660,27.726174,0.443215,27.726174,-1.026030,0.027007
9,DEN,24.825513,19.431284,0.565964,19.431284,-1.017416,0.020217


In [11]:
# Feature engineering: additional offensive metrics

# --- Average Yards Per Play ---

# Calculate mean yards gained per play at home and away
avg_yards_home = merged.groupby('Home')['Yards'].mean()
avg_yards_away = merged.groupby('Visitor')['Yards'].mean()

# Combine for team-wide average
avg_yards_per_play = (avg_yards_home + avg_yards_away) / 2

# --- Average Yards Per Game ---

# Sum total yards per team per season and divide by number of games played
total_yards_home = merged.groupby(['SeasonYear', 'Home'])['Yards'].sum() / merged.groupby(['SeasonYear', 'Home']).size()
total_yards_away = merged.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / merged.groupby(['SeasonYear', 'Visitor']).size()

# Combine to get team-wide season average
avg_yards_per_game = (total_yards_home + total_yards_away).groupby(level=1).mean()

# --- Pass Completion Rate ---

# Completion rate = 1 - incompletion rate
pass_comp_home = merged.groupby('Home')['IsIncomplete'].mean().apply(lambda x: 1 - x)
pass_comp_away = merged.groupby('Visitor')['IsIncomplete'].mean().apply(lambda x: 1 - x)

# Combine to get overall pass completion rate
avg_pass_completion = (pass_comp_home + pass_comp_away) / 2

# --- Touchdowns Per Game ---

# Touchdowns per game = total TDs divided by games played (per season)
tds_home = merged.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / merged.groupby(['SeasonYear', 'Home']).size()
tds_away = merged.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / merged.groupby(['SeasonYear', 'Visitor']).size()

# Combine for average TDs per game per team
avg_td_per_game = (tds_home + tds_away).groupby(level=1).mean()

# --- Rush Success Rate ---

# Rush success = average yards per rush
rush_success_home = merged[merged['IsRush'] == 1].groupby('Home')['Yards'].mean()
rush_success_away = merged[merged['IsRush'] == 1].groupby('Visitor')['Yards'].mean()

# Combine for team-wide average
avg_rush_success = (rush_success_home + rush_success_away) / 2

# --- Combine and Merge Offensive Features ---

# Create DataFrame of offensive metrics
offensive_features = pd.DataFrame({
    'Team': team_features_combined['Team'],
    'AvgYardsPerPlay': avg_yards_per_play.values,
    'AvgYardsPerGame': avg_yards_per_game.values,
    'AvgPassCompletionRate': avg_pass_completion.values,
    'AvgTouchdownsPerGame': avg_td_per_game.values,
    'AvgRushSuccessRate': avg_rush_success.values
})

# Merge into existing team feature set
team_features_expanded = team_features_combined.merge(offensive_features, on='Team')

# Preview final offensive features
team_features_expanded.head(32)


,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers,AvgYardsPerPlay,AvgYardsPerGame,AvgPassCompletionRate,AvgTouchdownsPerGame,AvgRushSuccessRate
0,ARI,21.939050,22.417655,0.419609,22.417655,-1.019879,0.021251,3.982764,7.965529,0.888556,0.052776,5.232888
1,ATL,22.178923,23.802673,0.504973,23.802673,-1.022210,0.023877,4.055938,8.111875,0.889247,0.057662,4.730858
2,BAL,30.215613,22.331169,0.679041,22.331169,-1.015552,0.017282,4.111842,8.223684,0.874797,0.068648,5.048079
3,BUF,32.132682,21.771034,0.810774,21.771034,-1.018904,0.020579,4.052990,8.105981,0.873419,0.076164,4.897583
4,CAR,18.581203,31.259658,0.260912,31.259658,-1.017999,0.018779,3.823520,7.647040,0.879671,0.063857,5.078586
5,CHI,17.595422,22.197156,0.244108,22.197156,-1.017277,0.017890,3.623948,7.247896,0.869852,0.052716,4.569097
6,CIN,28.780905,26.494829,0.501461,26.494829,-1.019098,0.022027,4.020278,8.040556,0.872226,0.078036,4.638758
7,CLE,15.724711,25.285987,0.185644,25.285987,-1.019512,0.021505,3.640863,7.281725,0.859458,0.054924,4.629079
8,DAL,20.873660,27.726174,0.443215,27.726174,-1.026030,0.027007,3.886588,7.773176,0.870121,0.061190,4.717464
9,DEN,24.825513,19.431284,0.565964,19.431284,-1.017416,0.020217,3.686734,7.373468,0.866159,0.053687,4.332698


In [12]:
# Feature engineering: additional defensive metrics

# --- Yards Allowed Per Play ---

# Calculate average yards allowed per play at home and away
yards_allowed_home = merged.groupby('Home')['Yards'].mean()
yards_allowed_away = merged.groupby('Visitor')['Yards'].mean()

# Combine to get total average yards allowed per team
avg_yards_allowed = (yards_allowed_home + yards_allowed_away) / 2

# --- Total Yards Allowed Per Game ---

# Sum total yards allowed per season and divide by number of games
total_yards_home = merged.groupby(['SeasonYear', 'Home'])['Yards'].sum() / merged.groupby(['SeasonYear', 'Home']).size()
total_yards_away = merged.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / merged.groupby(['SeasonYear', 'Visitor']).size()

# Combine and average over both sides
avg_yards_allowed_per_game = (total_yards_home + total_yards_away).groupby(level=1).mean()

# --- Pass Completion Allowed Rate ---

# Completion rate allowed = 1 - incompletion rate
comp_allowed_home = merged.groupby('Home')['IsIncomplete'].mean().apply(lambda x: 1 - x)
comp_allowed_away = merged.groupby('Visitor')['IsIncomplete'].mean().apply(lambda x: 1 - x)

# Combine for overall allowed completion rate
avg_pass_completion_allowed = (comp_allowed_home + comp_allowed_away) / 2

# --- Touchdowns Allowed Per Game ---

# Calculate touchdowns allowed per game from both sides
tds_allowed_home = merged.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / merged.groupby(['SeasonYear', 'Home']).size()
tds_allowed_away = merged.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / merged.groupby(['SeasonYear', 'Visitor']).size()

# Combine and average
avg_tds_allowed_per_game = (tds_allowed_home + tds_allowed_away).groupby(level=1).mean()

# --- Rush Success Allowed Rate ---

# Calculate average yards per rush allowed at home and away
rush_allowed_home = merged[merged['IsRush'] == 1].groupby('Home')['Yards'].mean()
rush_allowed_away = merged[merged['IsRush'] == 1].groupby('Visitor')['Yards'].mean()

# Combine for overall rush success rate allowed
avg_rush_success_allowed = (rush_allowed_home + rush_allowed_away) / 2

# --- Combine and Merge Defensive Features ---

# Create DataFrame of defensive metrics
defensive_features = pd.DataFrame({
    'Team': team_features_expanded['Team'],
    'AvgYardsAllowedPerPlay': avg_yards_allowed.values,
    'AvgYardsAllowedPerGame': avg_yards_allowed_per_game.values,
    'AvgPassCompletionAllowedRate': avg_pass_completion_allowed.values,
    'AvgTouchdownsAllowedPerGame': avg_tds_allowed_per_game.values,
    'AvgRushSuccessAllowedRate': avg_rush_success_allowed.values
})

# Merge with the previous feature set
team_features_complete = team_features_expanded.merge(defensive_features, on='Team')

# Preview the completed team feature dataset
team_features_complete.head(32)


,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers,AvgYardsPerPlay,AvgYardsPerGame,AvgPassCompletionRate,AvgTouchdownsPerGame,AvgRushSuccessRate,AvgYardsAllowedPerPlay,AvgYardsAllowedPerGame,AvgPassCompletionAllowedRate,AvgTouchdownsAllowedPerGame,AvgRushSuccessAllowedRate
0,ARI,21.939050,22.417655,0.419609,22.417655,-1.019879,0.021251,3.982764,7.965529,0.888556,0.052776,5.232888,3.982764,7.965529,0.888556,0.052776,5.232888
1,ATL,22.178923,23.802673,0.504973,23.802673,-1.022210,0.023877,4.055938,8.111875,0.889247,0.057662,4.730858,4.055938,8.111875,0.889247,0.057662,4.730858
2,BAL,30.215613,22.331169,0.679041,22.331169,-1.015552,0.017282,4.111842,8.223684,0.874797,0.068648,5.048079,4.111842,8.223684,0.874797,0.068648,5.048079
3,BUF,32.132682,21.771034,0.810774,21.771034,-1.018904,0.020579,4.052990,8.105981,0.873419,0.076164,4.897583,4.052990,8.105981,0.873419,0.076164,4.897583
4,CAR,18.581203,31.259658,0.260912,31.259658,-1.017999,0.018779,3.823520,7.647040,0.879671,0.063857,5.078586,3.823520,7.647040,0.879671,0.063857,5.078586
5,CHI,17.595422,22.197156,0.244108,22.197156,-1.017277,0.017890,3.623948,7.247896,0.869852,0.052716,4.569097,3.623948,7.247896,0.869852,0.052716,4.569097
6,CIN,28.780905,26.494829,0.501461,26.494829,-1.019098,0.022027,4.020278,8.040556,0.872226,0.078036,4.638758,4.020278,8.040556,0.872226,0.078036,4.638758
7,CLE,15.724711,25.285987,0.185644,25.285987,-1.019512,0.021505,3.640863,7.281725,0.859458,0.054924,4.629079,3.640863,7.281725,0.859458,0.054924,4.629079
8,DAL,20.873660,27.726174,0.443215,27.726174,-1.026030,0.027007,3.886588,7.773176,0.870121,0.061190,4.717464,3.886588,7.773176,0.870121,0.061190,4.717464
9,DEN,24.825513,19.431284,0.565964,19.431284,-1.017416,0.020217,3.686734,7.373468,0.866159,0.053687,4.332698,3.686734,7.373468,0.866159,0.053687,4.332698


In [13]:
# Game-level feature encoding for upcoming Week 1 matchups

# Reload upcoming games (redundant in Colab, but ensures fresh read if needed)
upcoming_games = pd.read_csv(UPCOMING_FILE)

# Merge home team features
upcoming_merged = upcoming_games.merge(
    team_features_complete,
    left_on='Home',
    right_on='Team',
    how='left'
)

# Merge visitor team features and suffix columns appropriately
upcoming_merged = upcoming_merged.merge(
    team_features_complete,
    left_on='Visitor',
    right_on='Team',
    how='left',
    suffixes=('_Home', '_Visitor')
)

# Preview encoded game matrix
upcoming_merged.head(16)

,Home,Visitor,Team_Home,AvgPointsScored_Home,AvgPointsAllowed_Home,WinRate_Home,AvgPointsDefended_Home,AvgConcededPlays_Home,AvgForcedTurnovers_Home,AvgYardsPerPlay_Home,AvgYardsPerGame_Home,AvgPassCompletionRate_Home,AvgTouchdownsPerGame_Home,AvgRushSuccessRate_Home,AvgYardsAllowedPerPlay_Home,AvgYardsAllowedPerGame_Home,AvgPassCompletionAllowedRate_Home,AvgTouchdownsAllowedPerGame_Home,AvgRushSuccessAllowedRate_Home,Team_Visitor,AvgPointsScored_Visitor,AvgPointsAllowed_Visitor,WinRate_Visitor,AvgPointsDefended_Visitor,AvgConcededPlays_Visitor,AvgForcedTurnovers_Visitor,AvgYardsPerPlay_Visitor,AvgYardsPerGame_Visitor,AvgPassCompletionRate_Visitor,AvgTouchdownsPerGame_Visitor,AvgRushSuccessRate_Visitor,AvgYardsAllowedPerPlay_Visitor,AvgYardsAllowedPerGame_Visitor,AvgPassCompletionAllowedRate_Visitor,AvgTouchdownsAllowedPerGame_Visitor,AvgRushSuccessAllowedRate_Visitor
0,PHI,DAL,PHI,27.784119,18.390556,0.807188,18.390556,-1.021505,0.022849,3.713618,7.427235,0.885785,0.062480,5.044430,3.713618,7.427235,0.885785,0.062480,5.044430,DAL,20.873660,27.726174,0.443215,27.726174,-1.026030,0.027007,3.886588,7.773176,0.870121,0.061190,4.717464,3.886588,7.773176,0.870121,0.061190,4.717464
1,LAC,KC,LAC,23.275070,17.615053,0.633606,17.615053,-1.016398,0.017415,3.737752,7.475503,0.872919,0.048244,4.743350,3.737752,7.475503,0.872919,0.048244,4.743350,KC,24.127831,18.013580,0.940591,18.013580,-1.018003,0.018656,3.710325,7.420649,0.872300,0.052486,4.402571,3.710325,7.420649,0.872300,0.052486,4.402571
2,NO,ARI,NO,19.658743,23.417726,0.311367,23.417726,-1.016530,0.018674,3.958671,7.917342,0.863787,0.057977,4.816401,3.958671,7.917342,0.863787,0.057977,4.816401,ARI,21.939050,22.417655,0.419609,22.417655,-1.019879,0.021251,3.982764,7.965529,0.888556,0.052776,5.232888,3.982764,7.965529,0.888556,0.052776,5.232888
3,NYJ,PIT,NYJ,19.241102,23.261580,0.247626,23.261580,-1.016311,0.018481,3.623839,7.247677,0.854772,0.056185,4.483378,3.623839,7.247677,0.854772,0.056185,4.483378,PIT,22.768932,20.283587,0.631544,20.283587,-1.021474,0.022367,3.763171,7.526341,0.880841,0.053082,4.335458,3.763171,7.526341,0.880841,0.053082,4.335458
4,IND,MIA,IND,22.046055,25.088614,0.438019,25.088614,-1.023501,0.025656,3.978577,7.957153,0.882756,0.062536,4.778634,3.978577,7.957153,0.882756,0.062536,4.778634,MIA,20.431246,20.660490,0.509738,20.660490,-1.018875,0.019877,3.698139,7.396277,0.880802,0.044901,4.452240,3.698139,7.396277,0.880802,0.044901,4.452240
5,ATL,TB,ATL,22.178923,23.802673,0.504973,23.802673,-1.022210,0.023877,4.055938,8.111875,0.889247,0.057662,4.730858,4.055938,8.111875,0.889247,0.057662,4.730858,TB,29.803002,23.249669,0.558920,23.249669,-1.023979,0.024978,4.110274,8.220547,0.880633,0.068720,5.119207,4.110274,8.220547,0.880633,0.068720,5.119207
6,WAS,NYG,WAS,28.506914,23.731939,0.680583,23.731939,-1.017854,0.018141,3.907699,7.815399,0.888713,0.068781,5.031666,3.907699,7.815399,0.888713,0.068781,5.031666,NYG,16.618699,24.434208,0.193538,24.434208,-1.018270,0.020052,3.713084,7.426168,0.874942,0.056458,4.874058,3.713084,7.426168,0.874942,0.056458,4.874058
7,JAX,CAR,JAX,18.676046,25.760421,0.241252,25.760421,-1.015473,0.016482,3.959279,7.918557,0.871454,0.057883,4.563894,3.959279,7.918557,0.871454,0.057883,4.563894,CAR,18.581203,31.259658,0.260912,31.259658,-1.017999,0.018779,3.823520,7.647040,0.879671,0.063857,5.078586,3.823520,7.647040,0.879671,0.063857,5.078586
8,CLE,CIN,CLE,15.724711,25.285987,0.185644,25.285987,-1.019512,0.021505,3.640863,7.281725,0.859458,0.054924,4.629079,3.640863,7.281725,0.859458,0.054924,4.629079,CIN,28.780905,26.494829,0.501461,26.494829,-1.019098,0.022027,4.020278,8.040556,0.872226,0.078036,4.638758,4.020278,8.040556,0.872226,0.078036,4.638758
9,NE,LV,NE,16.903046,25.333822,0.181087,25.333822,-1.021808,0.023020,3.655997,7.311995,0.873691,0.055492,4.759495,3.655997,7.311995,0.873691,0.055492,4.759495,LV,18.119686,25.006856,0.248521,25.006856,-1.017825,0.019077,3.590382,7.180764,0.867423,0.050676,4.314221,3.590382,7.180764,0.867423,0.05067

Data Training Preparation

In [14]:
# Training set construction: join game data with team features and build home-vs-visitor deltas

# Merge team features for the home side
training_encoded_home = merged.merge(
    team_features_complete,
    left_on='Home',
    right_on='Team',
    how='left'
)

# Merge team features for the visitor side (suffixes disambiguate feature columns)
training_encoded_both = training_encoded_home.merge(
    team_features_complete,
    left_on='Visitor',
    right_on='Team',
    how='left',
    suffixes=('_Home', '_Visitor')
)

# List the team-level feature names to difference (explicit for reproducibility)
diff_source_cols = [
    'AvgPointsScored', 'AvgPointsAllowed', 'WinRate',
    'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
    'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate',
    'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
    'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame',
    'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame',
    'AvgRushSuccessAllowedRate'
]

# Create home-minus-visitor differences for each feature
for col in diff_source_cols:
    training_encoded_both[f'Diff_{col}'] = (
        training_encoded_both[f'{col}_Home'] - training_encoded_both[f'{col}_Visitor']
    )

# Select model inputs (all Diff_ columns) and the target (home team win flag)
training_data = training_encoded_both[[c for c in training_encoded_both.columns if c.startswith('Diff_')]]
training_labels = training_encoded_both['HomeWon']

# Preview a sample of the finalized training matrix
training_data.head(50)

,Diff_AvgPointsScored,Diff_AvgPointsAllowed,Diff_WinRate,Diff_AvgPointsDefended,Diff_AvgConcededPlays,Diff_AvgForcedTurnovers,Diff_AvgYardsPerPlay,Diff_AvgYardsPerGame,Diff_AvgPassCompletionRate,Diff_AvgTouchdownsPerGame,Diff_AvgRushSuccessRate,Diff_AvgYardsAllowedPerPlay,Diff_AvgYardsAllowedPerGame,Diff_AvgPassCompletionAllowedRate,Diff_AvgTouchdownsAllowedPerGame,Diff_AvgRushSuccessAllowedRate
0,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.002050,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
1,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.002050,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
2,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.002050,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
3,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.002050,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
4,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.002050,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
5,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.002050,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
6,11.221799,-8.009988,0.298008,-8.009988,-0.005980,0.006200,0.286753,0.573507,0.000962,0.004863,0.040622,0.286753,0.573507,0.000962,0.004863,0.040622
7,11.221799,-8.009988,0.298008,-8.009988,-0.005980,0.006200,0.286753,0.573507,0.000962,0.004863,0.040622,0.286753,0.573507,0.000962,0.004863,0.040622
8,11.221799,-8.009988,0.298008,-8.009988,-0.005980,0.006200,0.286753,0.573507,0.000962,0.004863,0.040622,0.286753,0.573507,0.000962,0.004863,0.040622
9,11.221799,-8.009988,0.298008,-8.009988,-0.005980,0.006200,0.286753,0.573507,0.000962,0.004863,0.040622,0.286753,0.573507,0.000962,0.004863,0.040622


In [15]:
# Sanity checks: verify training matrix and labels before modeling

# Basic shapes
print("training_data shape:", training_data.shape)
print("training_labels shape:", training_labels.shape)

# Index alignment (ensures each row of X maps to the same row of y)
aligned = training_data.index.equals(training_labels.index)
print("Index aligned:", aligned)

# Missing values
total_feature_nas = int(training_data.isna().sum().sum())
label_nas = int(training_labels.isna().sum())
print("Total missing values in training_data:", total_feature_nas)
print("Missing labels:", label_nas)

# Type check: ensure all features are numeric
non_numeric_cols = training_data.select_dtypes(exclude=[np.number]).columns.tolist()
print("Non-numeric feature columns:", non_numeric_cols if non_numeric_cols else "None")

# Label distribution (helps catch class imbalance issues)
print("\nLabel distribution (HomeWon):")
print(training_labels.value_counts(dropna=False))

# Preview: show a small sample of the feature matrix
print("\nFeature columns (first 10):", list(training_data.columns[:10]))
training_data.head(10)

training_data shape: (48063, 16)
training_labels shape: (48063,)
Index aligned: True
Total missing values in training_data: 0
Missing labels: 0
Non-numeric feature columns: None

Label distribution (HomeWon):
HomeWon
True     25504
False    22559
Name: count, dtype: int64

Feature columns (first 10): ['Diff_AvgPointsScored', 'Diff_AvgPointsAllowed', 'Diff_WinRate', 'Diff_AvgPointsDefended', 'Diff_AvgConcededPlays', 'Diff_AvgForcedTurnovers', 'Diff_AvgYardsPerPlay', 'Diff_AvgYardsPerGame', 'Diff_AvgPassCompletionRate', 'Diff_AvgTouchdownsPerGame']


,Diff_AvgPointsScored,Diff_AvgPointsAllowed,Diff_WinRate,Diff_AvgPointsDefended,Diff_AvgConcededPlays,Diff_AvgForcedTurnovers,Diff_AvgYardsPerPlay,Diff_AvgYardsPerGame,Diff_AvgPassCompletionRate,Diff_AvgTouchdownsPerGame,Diff_AvgRushSuccessRate,Diff_AvgYardsAllowedPerPlay,Diff_AvgYardsAllowedPerGame,Diff_AvgPassCompletionAllowedRate,Diff_AvgTouchdownsAllowedPerGame,Diff_AvgRushSuccessAllowedRate
0,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.00205,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
1,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.00205,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
2,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.00205,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
3,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.00205,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
4,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.00205,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
5,-1.080438,-0.905936,0.211162,-0.905936,-0.001482,0.00205,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630,-0.167556,-0.335112,-0.012156,-0.004077,-0.491630
6,11.221799,-8.009988,0.298008,-8.009988,-0.005980,0.00620,0.286753,0.573507,0.000962,0.004863,0.040622,0.286753,0.573507,0.000962,0.004863,0.040622
7,11.221799,-8.009988,0.298008,-8.009988,-0.005980,0.00620,0.286753,0.573507,0.000962,0.004863,0.040622,0.286753,0.573507,0.000962,0.004863,0.040622
8,11.221799,-8.009988,0.298008,-8.009988,-0.005980,0.00620,0.286753,0.573507,0.000962,0.004863,0.040622,0.286753,0.573507,0.000962,0.004863,0.040622
9,11.221799,-8.009988,0.298008,-8.009988,-0.005980,0.00620,0.286753,0.573507,0.000962,0.004863,0.040622,0.286753,0.573507,0.000962,0.004863,0.040622


AI Model Training

In [16]:
# Model evaluation: logistic regression with k-fold cross-validation (no training changes)

# Imports repeated locally so this cell can run independently if needed
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Initialize the baseline logistic regression model
# Note: This is intentionally unchanged to keep parity with the original behavior
logreg = LogisticRegression(max_iter=1000)

# Evaluate the model using k-fold cross-validation
# Returns an array of scores (one per fold)
cross_val_scores = cross_val_score(
    estimator=logreg,
    X=training_data,
    y=training_labels,
    cv=10
)

# Compute the mean cross-validated score for quick comparison later
cross_val_scores_mean = cross_val_scores.mean()

# Display the mean CV score
cross_val_scores_mean

np.float64(0.7876323940127195)

In [17]:
# Model training: fit logistic regression on the full training dataset (unchanged behavior)

# Fit the model on all available training data after cross-validation
logreg.fit(training_data, training_labels)

# Optional: lightweight confirmation for visibility during refactors
print("Model fitted. Coefficients shape:", getattr(logreg, "coef_", None).shape if hasattr(logreg, "coef_") else None)

Model fitted. Coefficients shape: (1, 16)


Upcoming Game Predictions

In [18]:
# Upcoming games: construct home-minus-visitor feature deltas for Week 1 predictions

# Copy to avoid mutating the merged base
upcoming_features = upcoming_merged.copy()

# Use the same feature list used for training to avoid column drift
feature_cols = [f"Diff_{c}" for c in diff_source_cols]

# Create difference columns for each team-level feature
for col in diff_source_cols:
    home_col = f"{col}_Home"
    away_col = f"{col}_Visitor"
    diff_col = f"Diff_{col}"
    if home_col not in upcoming_features.columns or away_col not in upcoming_features.columns:
        raise KeyError(f"Required columns missing for upcoming games: '{home_col}' or '{away_col}'")
    upcoming_features[diff_col] = upcoming_features[home_col] - upcoming_features[away_col]

# Slice the finalized model input matrix for upcoming games
upcoming_X = upcoming_features[feature_cols].copy()

# Quick sanity checks before inference
print("Upcoming_X shape:", upcoming_X.shape)
print("Total missing values in upcoming_X:", int(upcoming_X.isna().sum().sum()))

# If there are NaNs, print which rows are affected to make debugging easier
if upcoming_X.isna().any().any():
    bad_rows = upcoming_X.index[upcoming_X.isna().any(axis=1)].tolist()
    print("Rows with missing values in upcoming_X:", bad_rows)

Upcoming_X shape: (16, 16)
Total missing values in upcoming_X: 0


In [19]:
# Inference: predict home-win probabilities for upcoming games and present results

# Predict probability that the home team wins (class 1)
upcoming_proba = logreg.predict_proba(upcoming_X)[:, 1]

# Attach predictions to the upcoming_features table
results = upcoming_features.copy()
results['Home Win Probability'] = upcoming_proba

# Determine predicted winner (threshold = 0.5 to match current model behavior)
results['Predicted Winner'] = results.apply(
    lambda r: r['Home'] if r['Home Win Probability'] >= 0.5 else r['Visitor'],
    axis=1
)

# Select a clean view for presentation
presentation_cols = ['Home', 'Visitor', 'Home Win Probability', 'Predicted Winner']
if 'Week' in results.columns:
    presentation_cols = ['Week'] + presentation_cols

# Sort by confidence in the home team (descending)
upcoming_predictions = results[presentation_cols].sort_values(by='Home Win Probability', ascending=False)

# Preview final predictions
upcoming_predictions

,Home,Visitor,Home Win Probability,Predicted Winner
6,WAS,NYG,0.949385,WAS
11,DEN,TEN,0.913147,DEN
0,PHI,DAL,0.903900,PHI
10,SEA,SF,0.742070,SEA
14,BUF,BAL,0.704238,BUF
12,LA,HOU,0.665189,LA
7,JAX,CAR,0.548492,JAX
5,ATL,TB,0.478116,TB
9,NE,LV,0.472943,LV
4,IND,MIA,0.468668,MIA
